In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists Lakehouse_Demo.Dim_Product
(
Product_ID	long	,
Product_Category string	,
Product	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  
)
using delta

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date = spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from Lakehouse_Demo.Dim_Product""").first()[0]
print(Max_Date)

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 5, Finished, Available, Finished)

1900-01-01


In [6]:
df_Sales = spark.sql("""select distinct 
Product_Category, 
Product 
from Lakehouse_Demo.Sales_Stg where Modified_TS>'{}'""".format(Max_Date))

display(df_Sales.limit(5))

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8240ce38-6e32-4493-8a1d-7d320d05a7a8)

In [10]:
Max_ID = spark.sql("select coalesce(max(Product_ID),0) from Lakehouse_Demo.Dim_Product").first()[0]

print(Max_ID)

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 12, Finished, Available, Finished)

0


In [12]:
df_final = df_Sales.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)
display(df_final.limit(5))

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 19897854-4614-49de-9bec-428e7dfda645)

In [13]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 15, Finished, Available, Finished)

In [17]:
%%sql
Merge into Lakehouse_Demo.Dim_Product as dp 
using ViewProduct as vp  
on dp.Product=vp.Product and dp.Product_Category=vp.Product_Category
when MATCHED then 
update set
Modified_TS= current_timestamp()

when not matched then
INSERT
(
Product_ID,
Product_Category,
Product,
Created_TS,
Modified_TS
)
VALUES
(
vp.Product_ID,
vp.Product_Category,
vp.Product,
current_timestamp(),
current_timestamp()

)

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 19, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [18]:
%%sql
select * from Lakehouse_Demo.Dim_Product

StatementMeta(, 9514791e-d20a-404e-ae96-b8220b1ee81f, 20, Finished, Available, Finished)

<Spark SQL result set with 42 rows and 5 fields>